Multiple passes of laser through crystal

In [ ]:
import sys, time
import math
import numpy as np
from pykern import pkcli
from pykern.pkcollections import PKDict

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import element
from rslaser.optics import drift
from rslaser.optics import crystal

import scipy.constants as const

import srwlib
from srwlib import srwl

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
%matplotlib inline

In [ ]:
# specify parameters
params = PKDict(
        photon_e_ev=1.5498, # Photon energy [eV], calculated from 800nm wavelength
        nslice = 50,
        nx_slice = 32,
        ny_slice = 32,
)

crystal_params = PKDict(length = 2.5/100.0,  # [m]
                        nslice = 6,
)

prop_type = 'gain_test'
num_propagate = 4

# Maximum value is num_propagate
prop_array = np.array([0, 4])

In [ ]:
e_crystal = crystal.Crystal(crystal_params)
nslices_crystal = len(e_crystal.slice)

# Assume all crystal slices have same cell volume
dx = (e_crystal.slice[0].pop_inversion_xfin-e_crystal.slice[0].pop_inversion_xstart)/e_crystal.slice[0].pop_inversion_nx
dy = (e_crystal.slice[0].pop_inversion_yfin-e_crystal.slice[0].pop_inversion_ystart)/e_crystal.slice[0].pop_inversion_ny
cell_volume = dx*dy*e_crystal.slice[0].length

thisPulse = pulse.LaserPulse(params)
nslices_pulse = len(thisPulse.slice)

photons_in_slice = np.zeros((nslices_pulse, num_propagate + 1))

for prop_index in np.arange(num_propagate + 1):
    
    for laser_index in np.arange(nslices_pulse):
        photons_in_slice[laser_index,prop_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d))
    
    total_excited_states = 0
    for crystal_index in np.arange(nslices_crystal):
        total_excited_states += np.sum(np.sum(e_crystal.slice[crystal_index].pop_inversion_mesh * cell_volume))
    print('Total number excited states in crystal, ', prop_index,' propagations: ', total_excited_states)

    if (prop_index < num_propagate):
        print('\nPropagating ', nslices_pulse, 'laser slices through ', nslices_crystal,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type)

# Assume all pulse slices have same length
fig_x = thisPulse.slice[0].ds *(np.arange(nslices_pulse) + 0.5)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
for prop_value in prop_array:
    plot_label = str(prop_value)+' propagations'
    plt.plot(fig_x,photons_in_slice[:,prop_value], label=plot_label)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)
ax.tick_params(direction="in")
ax.set_ylabel(r'Total Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(fig_x,photons_in_slice[:,0], 'k', label='Initialization')
ax2.plot(fig_x,photons_in_slice[:,4], '--r', label='4 Passes Through Crystal')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Distance from Laser Front [m]')
ax1.set_ylabel(r'Number of Photons, Initialized')
ax2.set_ylabel(r'Number of Photons, Propagated',color='r')
plt.title('Longitudinal Photon Distribution')